# Setting up

In [1]:
# Prepare dependencies
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('seaborn')

# Preparing datasets

In [2]:
legoData = pd.read_csv('lego2017_2020.csv')
legoData.head(2)

,SetID,Number,Variant,Theme,Subtheme,Year,Name,Minifigs,Pieces,USPrice,ImageURL,OwnedBy,WantedBy,Rating
0,29682,75267,1,Star Wars,The Mandalorian,2020,Mandalorian Battle Pack,4.0,102.0,14.99,https://images.brickset.com/sets/images/75267-...,2794,3080,NaN
1,29491,76895,1,Speed Champions,NaN,2020,Ferrari F8 Tributo,NaN,275.0,19.99,https://images.brickset.com/sets/images/76895-...,857,2055,NaN


In [3]:
legoData2 = pd.read_csv('lego_sets.csv')
legoData2.head(2)

,num_reviews,piece_count,play_star_rating,prod_desc,prod_id,review_difficulty,set_name,star_rating,theme_name,val_star_rating,country
0,2.0,277,4.0,Catapult into action and take back the eggs fr...,75823,Average,Bird Island Egg Heist,4.5,Angry Birds™,4.0,US
1,2.0,168,4.0,Launch a flying attack and rescue the eggs fro...,75822,Easy,Piggy Plane Attack,5.0,Angry Birds™,4.0,US


# Cleaning datasets

## 1. LegoData

In [4]:
# Make columns lowercase
legoData.columns = [x.lower()for x in legoData.columns]

# Revise column names
legoData = legoData.rename(columns={'imageurl': 'image_url',
                            'usprice': 'price',
                            'ownedby': 'owned_by',
                            'wantedby': 'wanted_by'})

In [24]:
# Check variables' types
legoData.dtypes

setid          int64
number        object
variant        int64
theme         object
subtheme      object
year           int64
name          object
pieces       float64
price        float64
image_url     object
owned_by       int64
wanted_by      int64
dtype: object

In [25]:
# Change type of 'setid'
legoData['setid'] = legoData['setid'].astype(str)

In [5]:
# Check NaNs
legoData.isnull().sum()

setid           0
number          0
variant         0
theme           0
subtheme      519
year            0
name            0
minifigs      562
pieces        108
price         460
image_url       0
owned_by        0
wanted_by       0
rating       1896
dtype: int64

In [6]:
# Remove 'Rating' & 'Minifigs' columns
legoData = legoData.drop(['rating', 'minifigs'], axis=1)

In [8]:
# Replace NaNs in 'subtheme' with 'Not Available'
legoData['subtheme'] = legoData['subtheme'].fillna('Not Available')

# Replace NaNs in 'pieces' with 0
legoData['pieces'] = legoData['pieces'].fillna(0)

# Replace NaNs in 'price' with 'pieces' * 0.08
# 0.08 is a rough-guess for purchasing a brick based on Lego website
legoData['price'] = legoData.apply(lambda row: row['pieces'] * 0.08 if np.isnan(row['price']) else row['price'],
                                   axis=1)

## 2. LegoData2

In [13]:
# Check unique info in 'country'
legoData2['country'].unique()

array(['US', 'AU', 'AT', 'BE', 'CA', 'CH', 'CZ', 'DE', 'DN', 'ES', 'FI',
       'FR', 'GB', 'IE', 'IT', 'LU', 'NO', 'NL', 'NZ', 'PL', 'PT'],
      dtype=object)

In [15]:
# Keep US data
legoData2 = legoData2.loc[legoData2['country'] == 'US']

In [27]:
# Check variables' types
legoData2.dtypes

num_reviews          float64
piece_count            int64
play_star_rating     float64
prod_desc             object
prod_id                int64
review_difficulty     object
set_name              object
star_rating          float64
theme_name            object
val_star_rating      float64
country               object
dtype: object

In [28]:
# Change type of 'prod_id'
legoData2['prod_id'] = legoData2['prod_id'].astype(str)

In [16]:
# Check NaNs
legoData2.isnull().sum()

num_reviews           98
piece_count            0
play_star_rating     107
prod_desc             20
prod_id                0
review_difficulty    124
set_name               0
star_rating           98
theme_name             0
val_star_rating      108
country                0
dtype: int64

In [17]:
# Replace NaNs in 'num_reviews', 'X_rating' with 0
legoData2['num_reviews'] = legoData2['num_reviews'].fillna(0)
legoData2['play_star_rating'] = legoData2['play_star_rating'].fillna(0)
legoData2['star_rating'] = legoData2['star_rating'].fillna(0)
legoData2['val_star_rating'] = legoData2['val_star_rating'].fillna(0)

# Replace NaN in 'review_difficulty', 'prod_desc' with 'Not Available'
legoData2['review_difficulty'] = legoData2['review_difficulty'].fillna('Not Available')
legoData2['prod_desc'] = legoData2['prod_desc'].fillna('Not Available')

## 3. Merged datasets

In [48]:
dfLego = pd.merge(legoData, 
                  legoData2,
                  left_on='number',
                  right_on='prod_id',
                  how='left')

In [54]:
# Remove duplicated columns ('piece_count', 'prod_id', 'theme_name')
dfLego = dfLego.drop(['piece_count', 'prod_id', 'theme_name'], axis=1)

# Remove 'country' column
dfLego = dfLego.drop(['country'], axis=1)

In [56]:
# Replace NaNs with 0 and 'Not Available'
dfLego['num_reviews'] = dfLego['num_reviews'].fillna(0)
dfLego['play_star_rating'] = dfLego['play_star_rating'].fillna(0)
dfLego['star_rating'] = dfLego['star_rating'].fillna(0)
dfLego['val_star_rating'] = dfLego['val_star_rating'].fillna(0)

dfLego['prod_desc'] = dfLego['prod_desc'].fillna("Not Available")
dfLego['review_difficulty'] = dfLego['review_difficulty'].fillna('Not Available')
dfLego['set_name'] = dfLego['set_name'].fillna('Not Available')

In [65]:
# Find deplicates
dfLego.duplicated('setid')

0       False
1       False
2       False
3       False
4       False
        ...  
2045    False
2046    False
2047     True
2048    False
2049    False
Length: 2050, dtype: bool

In [68]:
# Remove duplicates
dfLego = dfLego.drop_duplicates(['setid'])

# Exporting dataframe to a csv

In [69]:
# dfLego.to_csv ('lego_cleaned_set.csv', index = None, header=True)